In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import datetime as dt
import pandas as pd
import numpy as np

In [2]:
def get_web_page(url):
    resp = requests.get(url=url)
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        resp.encoding='utf-8'
        return resp.text    

# Start Crawer

In [3]:
start = 2012
end = 2019

url = 'https://www.playsport.cc/predictgame.php?action=result&allianceid=3&gametime='
f=open("/Users/chienan/Pycon/github/SportLottery/input/CRAWER.csv",'w')
f.write(u'\ufeff'.encode('utf8').decode('utf8'))
csv_writer = csv.writer(f)
        
csv_writer.writerow(['date','gameid'])
startdate = dt.datetime(start-1,10,1)
endate = dt.datetime(end,4,30)
totaldays = (endate - startdate).days + 1

In [4]:
doc = get_web_page("https://www.playsport.cc/predictgame.php?action=result&allianceid=3&gametime=20120310")

In [5]:
soup = BeautifulSoup(doc, "lxml")
data = soup.find('table',attrs={'class':'predictgame-table gamedata-results'})

In [6]:
team = data.findAll('td',attrs={'class':'td-teaminfo'})

In [7]:
start = 2012
end = 2019

url = 'https://www.playsport.cc/predictgame.php?action=result&allianceid=3&gametime='

startdate = dt.datetime(start-1,10,1)
endate = dt.datetime(end,4,30)
totaldays = (endate - startdate).days + 1
df = pd.DataFrame()

for daynumber in range(totaldays):
    datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y%m%d")
    url = 'https://www.playsport.cc/predictgame.php?action=result&allianceid=3&gametime='+datestring                                                     
    doc = get_web_page(url)
    soup = BeautifulSoup(doc, "lxml")
    data = soup.find('table',attrs={'class':'predictgame-table gamedata-results'})
    team = data.findAll('td',attrs={'class':'td-teaminfo'})
    
    for num in range(len(team)):
        game_date = (startdate + dt.timedelta(days = daynumber)).date()
        
        try:
            away_team = team[num].findAll('td')[1].text.strip()
            home_team = team[num].findAll('td')[3].text.strip()
        except:
            pass
        
        try:
            away_handicap_point = float(data.findAll('td',attrs={'class':'td-bank-bet01'})[num*2].findAll('strong')[1].text.strip())
            away_handicap_rate = float(data.findAll('td',attrs={'class':'td-bank-bet01'})[num*2].findAll('span')[1].text.replace(",","").strip())
        except:
            away_handicap_point = 0
            away_handicap_rate = 0
        
        try:
            home_handicap_point = float(data.findAll('td',attrs={'class':'td-bank-bet01'})[num*2+1].findAll('strong')[1].text.strip())
            home_handicap_rate = float(data.findAll('td',attrs={'class':'td-bank-bet01'})[num*2+1].findAll('span')[1].text.replace(",","").strip())
        except:
            home_handicap_point = 0
            home_handicap_rate = 0
        
        try:
            total_point =  float(float(data.findAll('td',attrs={'class':'td-bank-bet02'})[num*2].findAll('strong')[1].text.strip()))
            large_rate = float(data.findAll('td',attrs={'class':'td-bank-bet02'})[num*2].findAll('span')[1].text.replace(",","").strip())
            small_rate = float(data.findAll('td',attrs={'class':'td-bank-bet02'})[num*2+1].findAll('span')[1].text.replace(",","").strip())
        except:
            total_point = 0
            large_rate = 0
            small_rate = 0
        
        try:   
            away_rate = float(data.findAll('td',attrs={'class':'td-bank-bet03'})[num*2].findAll('span')[1].text.replace(",","").strip())
        except:
            away_rate = 0
        
        try:
            home_rate = float(data.findAll('td',attrs={'class':'td-bank-bet03'})[num*2+1].findAll('span')[1].text.replace(",","").strip())
        except:
            home_rate = 0
            
        df = pd.concat([df,pd.DataFrame(np.array([game_date,away_team,home_team,away_handicap_point,
                                                  home_handicap_point,away_handicap_rate,home_handicap_rate,
                                                  total_point,large_rate,small_rate,away_rate,
                                                  home_rate]).reshape(1,-1),
                                       columns=["GAME_DATE","AWAY","HOME","AWAY_HANDICAP_POINT","HOME_HANDICAP_POINT",
                                               "AWAY_HANDICAP_RATE","HOME_HANDICAP_RATE","TOTAL_POINT",
                                               "LARGE_RATE","SMALL_RATE","AWAY_RATE","HOME_RATE"])],axis=0)
    if (daynumber)%100==0:
        print("finish %d rows"%(daynumber+1))

finish 1 rows
finish 101 rows
finish 201 rows


SSLError: HTTPSConnectionPool(host='www.playsport.cc', port=443): Max retries exceeded with url: /predictgame.php?action=result&allianceid=3&gametime=20120611 (Caused by SSLError(SSLError("bad handshake: SysCallError(54, 'ECONNRESET')",),))

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.GAME_DATE.values[-1]

In [ ]:
df = df.loc[:9961,:]

In [26]:
df.to_pickle("/Users/chienan/Pycon/github/SportLottery/input/LOTTERY_CRAWER_2012to2019.pkl")

# Crawer for Team's Name

In [ ]:
# doc = get_web_page('https://quizlet.com/125064050/nba-flash-cards/')
# soup = BeautifulSoup(doc,"lxml")
# data = soup.findAll('span',attrs = {'class':'TermText'})    

In [ ]:
# team_name=[]
# for i in range(0,len(data),2):
#     team_name.append([data[i].text.strip(),data[i+1].text.strip()])

In [ ]:
# team_name = pd.DataFrame(team_name,columns=["TEAM_EN_NAME","TEAM_CH_NAME"])

In [ ]:
team_name = pd.read_pickle("/Users/chienan/Pycon/github/SportLottery/input/TEAMID_TRANS.pkl")

In [ ]:
df_ = df.merge(team_name, left_on="HOME", right_on="TEAM_CH_NAME",how="inner")
df_ = df_.merge(team_name, left_on="AWAY", right_on="TEAM_CH_NAME",how="inner",suffixes=["_HOME","_AWAY"])

In [ ]:
df_ = df_.sort_values("GAME_DATE").reset_index(drop=True)

In [ ]:
df_.to_pickle("LOTTERY_2012to2019.pkl")

In [ ]:
df = pd.DataFrame()
doc = get_web_page("https://www.playsport.cc/predictgame.php?action=result&allianceid=3&gametime=20120310")
soup = BeautifulSoup(doc, "lxml")
data = soup.find('table',attrs={'class':'predictgame-table gamedata-results'})
team = data.findAll('td',attrs={'class':'td-teaminfo'})
for num in range(len(team)):
    game_date = (startdate + dt.timedelta(days = daynumber)).date()
    away_team = team[num].findAll('td')[1].text.strip()
    home_team = team[num].findAll('td')[3].text.strip()
    
    away_handicap_point = float(data.findAll('td',attrs={'class':'td-bank-bet01'})[num*2].findAll('strong')[1].text.strip())
    home_handicap_point = float(data.findAll('td',attrs={'class':'td-bank-bet01'})[num*2+1].findAll('strong')[1].text.strip())
    away_handicap_rate = float(data.findAll('td',attrs={'class':'td-bank-bet01'})[num*2].findAll('span')[1].text.replace(",","").strip())
    home_handicap_rate = float(data.findAll('td',attrs={'class':'td-bank-bet01'})[num*2+1].findAll('span')[1].text.replace(",","").strip())
    
    total_point =  float(float(data.findAll('td',attrs={'class':'td-bank-bet02'})[num*2].findAll('strong')[1].text.strip()))
    large_rate = float(data.findAll('td',attrs={'class':'td-bank-bet02'})[num*2].findAll('span')[1].text.replace(",","").strip())
    small_rate = float(data.findAll('td',attrs={'class':'td-bank-bet02'})[num*2+1].findAll('span')[1].text.replace(",","").strip())
    
    away_rate = float(data.findAll('td',attrs={'class':'td-bank-bet03'})[num*2].findAll('span')[1].text.replace(",","").strip())
    home_rate = float(data.findAll('td',attrs={'class':'td-bank-bet03'})[num*2+1].findAll('span')[1].text.replace(",","").strip())
    df = pd.concat([df,pd.DataFrame(np.array([away_team,home_team,away_handicap_point,home_handicap_point,
                                              away_handicap_rate,home_handicap_rate,large_small_point,
                                              large_rate,small_rate,away_rate,home_rate]).reshape(1,-1),
                                   columns=["AWAY","HOME","AWAY_HANDICAP_POINT","HOME_HANDICAP_POINT",
                                           "AWAY_HANDICAP_RATE","HOME_HANDICAP_POINT","TOTAL_POINT",
                                           "LARGE_RATE","SMALL_RATE","AWAY_RATE","HOME_RATE"])],axis=0)


In [27]:
doc = get_web_page("https://www.playsport.cc/predictgame.php?action=result&allianceid=3&gametime=20120310")
soup = BeautifulSoup(doc, "lxml")
data = soup.find('table',attrs={'class':'predictgame-table gamedata-results'})
team = data.findAll('td',attrs={'class':'td-teaminfo'})

In [12]:
data.findAll('td',attrs={'class':'td-bank-bet01'})[3].findAll('strong')[1].text.strip()

'185.5'

In [33]:
data.findAll('td',attrs={'class':'td-bank-bet03'})[1]

<td class="td-bank-bet03">
<div>
<strong class="team-side DATA_MID_CLASS">主</strong><span class="data-wrap"><strong></strong><span>1.99</span></span> </div>
</td>

In [62]:
doc = get_web_page("https://www.sportslottery.com.tw/zh/web/guest/sports-betting#m/all/s-442/c/t-4102/278/1")
soup = BeautifulSoup(doc, "lxml")

In [64]:
soup.findAll("div")

[<div class="portlet-boundary portlet-boundary_bonRootPortlet_WAR_taiwanplayerclubportlet_ portlet-static portlet-static-end " id="p_p_id_bonRootPortlet_WAR_taiwanplayerclubportlet_"> <a id="p_bonRootPortlet_WAR_taiwanplayerclubportlet"></a> <div class="portlet-borderless-container" style=""> <div class="portlet-body"> <script>/*<![CDATA[*/PlayerClub.handleNoInternetConnection=function(a){hideSpinner();if(Validator.Scanner.lastContext){Validator.Scanner.add(Validator.Scanner.lastContext);Validator.Scanner.lastContext.enableButton()}if(a){a.oncomplete=undefined}if($.mobile){var c=$(".ui-popup-active").find("[data-role=popup]");if(c.attr("id")!="noInternetConnectionMobileDialog"){var b=$("#noInternetConnectionMobileDialog");b.find(".btn-red, .popup-close").off("click");if(c!=undefined&&c.length>0){c.on({popupafterclose:function(){b.popup("open");$(this).off("popupafterclose")}})}b.find(".btn-red, .popup-close").on("click",function(d){d.preventDefault();d.stopPropagation();b.popup("close"

In [ ]:
data = soup.find('table',attrs={'class':'predictgame-table gamedata-results'})
team = data.findAll('td',attrs={'class':'td-teaminfo'})